In [132]:
import sys
import os 
import pandas as pd
import numpy as np

import soepy

from dev_library import plot_basics_choices
from dev_library import plot_basics_wages
from dev_library import df_alignment

import pandas as pd
pd.set_option('display.max_rows', 500)
sys.path.insert(0, os.environ["PROJECT_DIR"] + "/estimations/peisenha_exploration")

In [98]:
fname = os.environ["PROJECT_DIR"] + "/resources/soepcore_struct_prep.dta"
df_obs = pd.read_stata(fname, convert_categoricals = False)
df_obs = df_alignment(df_obs, is_obs=True)

In [99]:
df_obs["Age_Youngest_Child"].value_counts(sort=True).sort_index()

-1.0     41773
 0.0      2600
 1.0      3218
 2.0      2982
 3.0      2942
 4.0      2634
 5.0      2413
 6.0      2285
 7.0      2264
 8.0      2202
 9.0      1989
 10.0     1938
Name: Age_Youngest_Child, dtype: int64

In [155]:
bins

IntervalIndex([(-0.1, 2.1], (2.9, 5.1], (5.9, 10.1]],
              closed='right',
              dtype='interval[float64]')

In [138]:
bins = pd.IntervalIndex.from_tuples([(-0.1, 2.1), (2.9, 5.1), (5.9, 10.1)]) 
labels = ['0-2', '3-5', '6-10']
df_obs["Age_Range"] = pd.cut(df_obs["Age_Youngest_Child"] , bins, labels = labels)

In [139]:
df_obs["Age_Range"].value_counts()

(5.9, 10.1]    10678
(-0.1, 2.1]     8800
(2.9, 5.1]      7989
Name: Age_Range, dtype: int64

In [140]:
2600 + 3218 + 2982  

8800

In [141]:
2942 + 2634 + 2413

7989

In [142]:
2285 + 2264 + 2202 + 1989 + 1938

10678

In [168]:
LABELS_AGE = ['0-2', '3-5', '6-10']
LABELS_EDUCATION = ["High", "Medium", "Low"]
LABELS_CHOICE = ["Home", "Part", "Full"]
LABELS_WORK = ["Part", "Full"]


def get_moments(df):


    df_int = df.copy()

    # For the observed dataset, we have many missing values in our dataset and so we must
    # restrict attention to those that work and make sure we have a numeric type.
    df_sim_working = df_int[df_int["Choice"].isin(LABELS_WORK)]
    df_sim_working = df_sim_working.astype({"Wage_Observed": np.float})

    # We need to add information on the age range of the youngest child.
    bins = pd.IntervalIndex.from_tuples([(-0.1, 2.1), (2.9, 5.1), (5.9, 10.1)]) 
    df_int["Age_Range"] = pd.cut(df_int["Age_Youngest_Child"], bins, labels=LABELS_AGE)
    
    num_periods = df_int.index.get_level_values("Period").max()

    # Choice probabilities, differentiating by age range of youngest child, default entry is zero
    # We restrict attention to the first 20 periods as aferwards the cells get rather thin
    max_period = 20
    entries = [list(range(max_period)), bins.get_level_values(0), LABELS_CHOICE]
    conditioning = ["Period", "Age_Range", "Choice"]
    default_entry = 0

    index = pd.MultiIndex.from_product(entries, names=conditioning)
    df_probs_grid = pd.DataFrame(data=default_entry, columns=["Value"], index=index)

    
    df_probs = df_int.groupby(conditioning[:2]).Choice.value_counts(normalize=True).rename("Value")
    df_probs_grid.update(df_probs)
    
    moments = list(df_probs_grid.sort_index().values.flatten())

    
    return df_probs_grid

df_probs_grid = get_moments(df_obs)

In [169]:
df_probs_grid

Value
Period Age_Range   Choice          
0      (-0.1, 2.1] Home    0.892157
                   Part    0.029412
                   Full    0.078431
       (2.9, 5.1]  Home    0.705882
                   Part    0.117647
                   Full    0.176471
       (5.9, 10.1] Home    1.000000
                   Part    0.000000
                   Full    0.000000
1      (-0.1, 2.1] Home    0.918919
                   Part    0.054054
                   Full    0.027027
       (2.9, 5.1]  Home    0.744681
                   Part    0.148936
                   Full    0.106383
       (5.9, 10.1] Home    0.777778
                   Part    0.111111
                   Full    0.111111
2      (-0.1, 2.1] Home    0.849765
                   Part    0.084507
                   Full    0.065728
       (2.9, 5.1]  Home    0.686047
                   Part    0.232558
                   Full    0.081395
       (5.9, 10.1] Home    0.611111
                   Part    0.333333
                   Full    0.055556
3      (-0.1, 2.1] Home    0.868421
                   Part    0.060150
                   Full    0.071429
       (2.9, 5.1]  Home    0.805556
                   Part    0.157407
                   Full    0.037037
       (5.9, 10.1] Home    0.454545
                   Part    0.393939
                   Full    0.151515
4      (-0.1, 2.1] Home    0.864078
                   Part    0.077670
                   Full    0.058252
       (2.9, 5.1]  Home    0.714286
                   Part    0.246032
                   Full    0.039683
       (5.9, 10.1] Home    0.621622
                   Part    0.270270
                   Full    0.108108
5      (-0.1, 2.1] Home    0.834734
                   Part    0.098039
                   Full    0.067227
       (2.9, 5.1]  Home    0.716312
                   Part    0.241135
                   Full    0.042553
       (5.9, 10.1] Home    0.687500
                   Part    0.125000
                   Full    0.187500
6      (-0.1, 2.1] Home    0.830918
                   Part    0.101449
                   Full    0.067633
       (2.9, 5.1]  Home    0.671362
                   Part    0.215962
                   Full    0.112676
       (5.9, 10.1] Home    0.594937
                   Part    0.240506
                   Full    0.164557
7      (-0.1, 2.1] Home    0.800000
                   Part    0.111579
                   Full    0.088421
       (2.9, 5.1]  Home    0.601594
                   Part    0.286853
                   Full    0.111554
       (5.9, 10.1] Home    0.439655
                   Part    0.327586
                   Full    0.232759
8      (-0.1, 2.1] Home    0.786667
                   Part    0.133333
                   Full    0.080000
       (2.9, 5.1]  Home    0.546053
                   Part    0.325658
                   Full    0.128289
       (5.9, 10.1] Home    0.445161
                   Part    0.329032
                   Full    0.225806
9      (-0.1, 2.1] Home    0.763727
                   Part    0.149750
                   Full    0.086522
       (2.9, 5.1]  Home    0.505780
                   Part    0.346821
                   Full    0.147399
       (5.9, 10.1] Home    0.336788
                   Part    0.419689
                   Full    0.243523
10     (-0.1, 2.1] Home    0.790850
                   Part    0.140523
                   Full    0.068627
       (2.9, 5.1]  Home    0.515366
                   Part    0.378251
                   Full    0.106383
       (5.9, 10.1] Home    0.457692
                   Part    0.330769
                   Full    0.211538
11     (-0.1, 2.1] Home    0.765528
                   Part    0.159938
                   Full    0.074534
       (2.9, 5.1]  Home    0.502913
                   Part    0.380583
                   Full    0.116505
       (5.9, 10.1] Home    0.430723
                   Part    0.364458
                   Full    0.204819
12     (-0.1, 2.1] Home    0.758347
                   Part    0.16693